In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_21339/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_21339/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_48_5,0.993331,0.807742,0.780249,0.983675,0.027760,1.285627,0.786037,0.076668,0.306767,0.166613,1.426818,0.169237,137.168319,278.503492,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_48_3,0.993514,0.807088,0.779897,0.985519,0.026997,1.290002,0.787295,0.068009,0.303549,0.164308,1.415087,0.166895,137.224058,278.559231,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_48_0,0.993604,0.806407,0.780093,0.987429,0.026624,1.294555,0.786594,0.059038,0.300304,0.163167,1.409345,0.165737,137.251917,278.587090,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_49_2,0.994404,0.805840,0.782904,0.998252,0.023295,1.298351,0.776538,0.012591,0.265444,0.152626,1.358163,0.155030,137.519053,278.854226,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_49_6,0.994674,0.805163,0.783697,0.997987,0.022168,1.302878,0.773704,0.014501,0.257112,0.148890,1.340845,0.151235,137.618178,278.953350,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,model_1_6_3,0.926668,0.714172,0.773652,0.838501,0.305246,1.911334,0.809635,0.484066,0.918409,0.552490,5.693235,0.561191,132.373277,273.708450,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
131,model_1_6_0,0.926952,0.712340,0.773470,0.869629,0.304065,1.923584,0.810285,0.390764,0.865435,0.551421,5.675084,0.560105,132.381027,273.716200,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
132,model_1_5_7,0.898942,0.677981,0.755287,0.943199,0.420655,2.153340,0.875323,0.277819,0.853796,0.648579,7.467688,0.658794,131.731884,273.067057,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
133,model_1_5_3,0.890199,0.669240,0.745981,0.937081,0.457051,2.211795,0.908611,0.307743,0.886785,0.676055,8.027282,0.686702,131.565922,272.901094,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
